#Final Project - Sequence Generation with Keras
####Matan Zubery - 316263938  Ohad Dali - 316452929

Our mission is to choose genomes, learn a model for each gender and generate new genomes based on the models.
The genomes is taken from [www.internationalgenome.org/data](http://www.internationalgenome.org/data).

The sequence : **genome** ,The type of the sequence : **male or female**


##Part A - Data Collection
The data is stored in FASTQ format. FASTQ is a text-based format for storing a biological squence and its corresponding quality scores. Both the sequence letter and quality score are each encoded with a single ASCII character for brevity.

We will need to use the biopython library to parse the files and read it properly.

In [87]:
from Bio import SeqIO
from Bio.Seq import Seq

The samples that I used is stored in large files, so for uploading the samples to Github , I will make small samples of each genome.

In [88]:
from os import listdir
from os.path import isfile,join

In [89]:

genomesFiles = [f for f in listdir("data\male") if isfile(join("data\male",f))]
for file in genomesFiles:
    with open("data/male/"+file) as source:
        with open("data/male/Small Samples/"+file, "w") as f1:
            limit = 100000
            index = 1
            for line in source:
                    f1.write(line)
                    if index == limit:
                        break
                    index = index + 1
                    
genomesFiles = [f for f in listdir("data/female") if isfile(join("data/female",f))]
for file in genomesFiles:
    with open("data/female/"+file) as source:
        with open("data/female/Small Samples/"+file, "w") as f1:
            limit = 100000
            index = 1
            for line in source:
                    f1.write(line)
                    if index == limit:
                        break
                    index = index + 1

Now, let's examine this data and recognize some issues to handle.

In [4]:
records = SeqIO.parse("data\male\Small Samples\SRR101470_2.fastq","fastq")
first_record = records.next()
print first_record
print "seq len: " + str(len(first_record.seq))

ID: SRR101470.1
Name: SRR101470.1
Description: SRR101470.1 61C53AAXX100218:2:1:995:8481/2
Number of features: 0
Per letter annotation for: phred_quality
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN', SingleLetterAlphabet())
seq len: 76


We can see that each record holds a part of the whole genom.
So we need to group those records into whole genom (at least part of it).


In [5]:
genome = first_record.seq
for record in records:
    genome = genome + record.seq

KeyboardInterrupt: 

Now let's have a look at the sample as raw text.

In [13]:
print len(genome)
genome

1900000


Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...AGC', SingleLetterAlphabet())

The DNA is built from 4 bases : A T C G . The N we see in our genom means that the sequencer was not able to determine if the base was A,T,C or G so this read is useless.


In [23]:
genome1 = str(genome)
genome1 = genome1.replace('N','')
abc = Seq(genome1)
Seq(genome1)

Seq('CTTTCTCCAAAGCTGGCGCATAAGACCAACAAAGCACTGAATCGAACTGGTAGT...AGC', Alphabet())

Now we do it to the rest of the genome samples

In [90]:
MaleGenoms = []
Malesamples = [f for f in listdir("data\male\Small Samples") if isfile(join("data\male\Small Samples",f))]
for file in Malesamples:
    records = SeqIO.parse("data/male/Small Samples/"+file,"fastq")
    genome = records.next().seq;
    for record in records:
        genome = genome + record.seq
    genome1 = str(genome).replace('N','')
    MaleGenoms.append(Seq(genome1))

Same to the female samples:

In [91]:
FemaleGenoms = []
index = 1
Femalesamples = [f for f in listdir("data/female\Small Samples") if isfile(join("data/female\Small Samples",f))]
for file in Femalesamples:
    records = SeqIO.parse("data/female/Small Samples/"+file,"fastq")
    genome = records.next().seq
    for record in records:
        genome = genome + record.seq  
    genome1 = str(genome).replace('N','')
    FemaleGenoms.append(Seq(genome1))

For this mission I  took 6 samples of males and 6 samples of females.
For each gender, I used 2 samples that was taken from British people, 2 samples that was taken from Finnish pepole and 2 samples that was taken from Japanese people.

The size of each sample is more than 1.5 giga (after compressing!), and download sufficient quantity of samples and unzip it will take me more than doing this assignmet. So I decided to split the 12 samples I already downloaded.

In [92]:
sampleLength = 50000
Male_Small_Genomes = []
for bigSample in MaleGenoms:
    sampleAsStr = str(bigSample)
    smallSamples = [sampleAsStr[i:i + sampleLength] for i
                    in range(0 , len(bigSample) , sampleLength)]
    Male_Small_Genomes =Male_Small_Genomes + smallSamples

len(Male_Small_Genomes)

255

In [93]:
Female_Small_Genomes = []
for bigSample in FemaleGenoms:
    sampleAsStr = str(bigSample)
    smallSamples = [sampleAsStr[i:i + sampleLength] for i
                    in range(0 , len(bigSample) , sampleLength)]
    Female_Small_Genomes =Female_Small_Genomes + smallSamples

len(Female_Small_Genomes)

218

Now we have 255 samples of man gemome and 218 samples of woman genome

##Part B - Building the Classifier



First, we need to find a way to repersent the collected data as a vector of features. 

The feature that first came in our mind is base frequency.
For each sequence we will count the occurrences of each base(A , T , G and C). 
After that we divide by the sequence length, because the sequences are in different lengths. 


We will construct an empty dictionary. Each row in the dictionary will be the vector features of each sequence. 

In [97]:
dict = {}
for fg in Female_Small_Genomes:
    dict[fg] = []
for mg in Male_Small_Genomes:
    dict[mg] = []

Forcing the division to be floating point:

In [98]:
from __future__ import  division


Calculating the base frequencies features, first for the ladies.

In [99]:

for genome in Female_Small_Genomes:
    length = len(genome)
    AFreq = str(genome).count('A')/length
    TFreq = str(genome).count('T')/length
    GFreq = str(genome).count('G')/length
    CFreq = str(genome).count('C')/length
    dict[genome].append(AFreq)
    dict[genome].append(TFreq)
    dict[genome].append(GFreq)
    dict[genome].append(CFreq)
    
    
for genome in Male_Small_Genomes:
    length = len(genome)
    AFreq = str(genome).count('A')/length
    TFreq = str(genome).count('T')/length
    GFreq = str(genome).count('G')/length
    CFreq = str(genome).count('C')/length
    dict[genome].append(AFreq)
    dict[genome].append(TFreq)
    dict[genome].append(GFreq)
    dict[genome].append(CFreq)


Adding the gender feature.

In [100]:
for fg in Female_Small_Genomes:
    dict[fg].append(0) # 0 for female
for mg in Male_Small_Genomes:
    dict[mg].append(1) # 1 for male

Let's build a dataframe using pandas library

In [101]:
import pandas as pd
df = pd.DataFrame.from_dict(dict,orient="columns").T
df.columns = ['A_FREQ','T_FREQ','G_FREQ','C_FREQ','Gender']


Describing the dataframe to see if there is any differnce between the genders.

In [102]:
df.shape

(473, 5)

In [103]:
groupby_gender = df.groupby('Gender') 
groupby_gender.mean()

,A_FREQ,T_FREQ,G_FREQ,C_FREQ
Gender,,,,
0.0,0.280712,0.274006,0.222271,0.223010
1.0,0.287756,0.284773,0.214633,0.212838


It seems that there is no noticable difference between male sequences and female sequences. we need to find more features.

In [104]:
gen1 = 'ATGC'
gen2 = 'AGGC'
gen3 = 'TACG'

for genome in Female_Small_Genomes:
    Gen1Freq = str(genome).count(gen1)
    Gen2Freq = str(genome).count(gen2) 
    Gen3Freq = str(genome).count(gen3)
    dict[genome].append(Gen1Freq)
    dict[genome].append(Gen2Freq)
    dict[genome].append(Gen3Freq)
    
for genome in Male_Small_Genomes:
    Gen1Freq = str(genome).count(gen1)
    Gen2Freq = str(genome).count(gen2) 
    Gen3Freq = str(genome).count(gen3)
    dict[genome].append(Gen1Freq)
    dict[genome].append(Gen2Freq)
    dict[genome].append(Gen3Freq)

In [105]:
df = pd.DataFrame.from_dict(dict,orient="columns").T
df.columns = ['A_FREQ','T_FREQ','G_FREQ','C_FREQ','Gender','Gen1_Quan','Gen2_Quan','Gen3_Quan']

In [106]:
groupby_gender = df.groupby('Gender') 
groupby_gender.mean()

,A_FREQ,T_FREQ,G_FREQ,C_FREQ,Gen1_Quan,Gen2_Quan,Gen3_Quan
Gender,,,,,,,
0.0,0.280712,0.274006,0.222271,0.223010,172.013761,228.050459,32.266055
1.0,0.287756,0.284773,0.214633,0.212838,170.952941,209.070588,33.458824


In [107]:

index = 0
for key,row in df.iterrows():
    df.iloc[index, df.columns.get_loc('Gender')] =  'Female' if row['Gender'] == 0 else 'Male'
    index = index + 1 


In [111]:
import numpy as np
msk = np.random.rand(len(df)) < 0.7

train = df[msk]
test = df[~msk]

trainX = train[['A_FREQ','T_FREQ','G_FREQ','C_FREQ','Gen1_Quan','Gen2_Quan','Gen3_Quan']]
trainY = train['Gender']


testX = test[['A_FREQ','T_FREQ','G_FREQ','C_FREQ','Gen1_Quan','Gen2_Quan','Gen3_Quan']]
testY = test['Gender']

len(test)

154

In [112]:
from sklearn import svm

# Initialize a svc classifier with gamma = 0.001, C = 100, degree = 3
svc = svm.SVC(gamma = 0.001, C = 100, degree = 3)

# Fit the svc to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
svc = svc.fit(trainX, trainY)

# Evaluate accuracy best on the test set
support_vector = svc.score(testX,testY)

support_vector

0.6363636363636364

In [113]:
# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( trainX, trainY )

# Evaluate accuracy best on the test set
random_forest = forest.score(testX,testY)

random_forest

0.7662337662337663

##Building the network
####Setup

In [157]:
import numpy as np
import theano
import keras
from keras import backend as k
k.set_image_dim_ordering('th')
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

In [194]:
from keras.preprocessing.text import text_to_word_sequence
maleGenomes = df.index[df['Gender'] == 'Male'].tolist()
maleGenomes = [" ".join(str(g)) for g in maleGenomes] #Every base seperated by space


In [ ]:

maleGenomes = " NEWSEQ ".join(maleGenomes)
maleInputSplitted = text_to_word_sequence(maleGenomes, lower=False, split=' ')

In [215]:
maleInputSplitted.index(" NEWSEQ ")

ValueError: ' NEWSEQ ' is not in list

In [178]:
from keras.preprocessing.text import Tokenizer
male_tokenizer = Tokenizer(num_words=4,char_level=False)
male_tokenizer.fit_on_texts(maleInputSplitted)

In [179]:
male_text_mtx = male_tokenizer.texts_to_matrix(maleInputSplitted, mode='binary')
male_input = male_text_mtx[:-1]
male_output = male_text_mtx[1:]
#vocab = pd.DataFrame({'word':maleInputSplitted,'code':np.argmax(male_text_mtx,axis=1)})


In [180]:
malemodel = Sequential()
malemodel.add(Embedding(input_dim=male_input.shape[1],output_dim= 42, input_length=male_input.shape[1]))

malemodel.add(Flatten())
malemodel.add(Dense(male_output.shape[1], activation='sigmoid'))

malemodel.summary(line_length=100)

len(male_input)

____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
embedding_5 (Embedding)                      (None, 4L, 42)                          168            
____________________________________________________________________________________________________
flatten_5 (Flatten)                          (None, 168)                             0              
____________________________________________________________________________________________________
dense_5 (Dense)                              (None, 4L)                              676            
Total params: 844
Trainable params: 844
Non-trainable params: 0
____________________________________________________________________________________________________


12653281

In [154]:
malemodel.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])

In [155]:
malemodel.fit(male_input, y=male_output, batch_size=200, epochs=2, verbose=1, validation_split=0.3)

Train on 8857122 samples, validate on 3795910 samples
Epoch 1/2


    200/8857122 [..............................] - ETA: 16:11:18 - loss: 1.1288 - acc: 0.2700

    400/8857122 [..............................] - ETA: 9:09:29 - loss: 1.1264 - acc: 0.2475 

    600/8857122 [..............................] - ETA: 6:40:01 - loss: 1.1148 - acc: 0.2517

    800/8857122 [..............................] - ETA: 5:29:31 - loss: 1.1018 - acc: 0.2400

...

 494800/8857122 [>.............................] - ETA: 1:17:09 - loss: 0.8460 - acc: 0.3245

 495000/8857122 [>.............................] - ETA: 1:17:09 - loss: 0.8460 - acc: 0.3245

 495200/8857122 [>.............................] - ETA: 1:17:09 - loss: 0.8459 - acc: 0.3245

 495400/8857122 [>.............................] - ETA: 1:17:09 - loss: 0.8459 - acc: 0.3246

KeyboardInterrupt: 

In [124]:
male_score = malemodel.evaluate(male_input,male_output, verbose=0)
print('male_score : ', male_score)

('male_score : ', [0.1427690087575612, 0.5])


In [125]:
def get_next(text,token,model,vocabulary):
    '''Predicts the following word, given a text word, a tokenizer to convert it to 1-hot vector, a trained model and a vocabulary
    with word and index representations'''
    #converting the word to 1-hot matrix represenation
    tmp = text_to_word_sequence(text, lower=False, split=" ")
    tmp = token.texts_to_matrix(tmp, mode='binary')
    #predicting next word
    bestMatch=model.predict_classes(tmp)[0]
    return vocabulary[vocabulary['code']==bestMatch]['word'].values[0]

In [126]:
get_next(" NEW ",male_tokenizer,malemodel,vocab)

'NEW'

In [134]:
get_next("G",male_tokenizer,malemodel,vocab)

'NEW'

In [128]:
def create_sentence(token, model, vocab):
    prev_word = ' NEW '
    next_word = ''
    res = '';
    count_words = 0
    maxSentenceLength = 10
    while next_word.strip() != ' NEW ' and count_words < maxSentenceLength:
        next_word = get_next(prev_word, token, model, vocab)
        res = res + " " + next_word
        prev_word = next_word
        count_words += 1
    return res.rsplit(' ', 1)[0]

In [129]:
create_sentence(male_tokenizer,malemodel,vocab)

' NEW NEW NEW NEW NEW NEW NEW NEW NEW'

In [ ]:
get_next()